In [1]:
import os
import re
import requests
import pandas as pd
import geopandas as gpd

# (Recommended) set a Socrata app token to reduce throttling
# os.environ["SOCRATA_APP_TOKEN"] = "YOUR_TOKEN_HERE"

# ---- NYC Open Data dataset IDs ----
DS_TREE_POINTS = "hn5i-inap"  # Forestry Tree Points (street-tree planting spaces / trees)
DS_WORK_ORDERS = "bdjm-n7q4"  # Forestry Work Orders (events: removals, plantings, etc.)
DS_CENSUS_2015 = "uvpi-gqnh"  # (optional enrichment) 2015 Street Tree Census

# Optional: Census Tracts via API if you want to tag tracts (fill with the correct dataset id later)
CT_DATASET_ID  = None  # e.g. "c7iz-8g3q" (example only)

# ---- Analysis window ----
YEARS = list(range(2010, 2018))  # 2010..2017 inclusive
AS_OF = {y: pd.Timestamp(f"{y}-12-31") for y in YEARS}

# Spatial CRS for NYC (planar feet) if you do spatial joins later
TARGET_CRS = "EPSG:2263"


In [2]:
BASE = "https://data.cityofnewyork.us/resource/{id}"

def _headers():
    h = {}
    tok = os.getenv("SOCRATA_APP_TOKEN")
    if tok:
        h["X-App-Token"] = tok
    return h

def socrata_fetch_tabular(dataset_id, where=None, select=None, limit=25000, max_rows=None, verbose=True):
    """
    Pull ALL rows (paged) from a Socrata tabular dataset (.json). Returns a DataFrame.
    Set max_rows=None to attempt full download.
    """
    url = f"{BASE.format(id=dataset_id)}.json"
    params_base = {"$limit": limit}
    if where:  params_base["$where"]  = where
    if select: params_base["$select"] = select

    frames, offset, got = [], 0, 0
    while True:
        if max_rows is not None and got >= max_rows:
            break
        page = min(limit, max_rows - got) if max_rows else limit
        p = dict(params_base); p["$offset"] = offset; p["$limit"] = page
        r = requests.get(url, params=p, headers=_headers(), timeout=180); r.raise_for_status()
        rows = r.json()
        if not rows: break
        frames.append(pd.DataFrame(rows))
        n = len(rows); got += n; offset += n
        if verbose: print(f"{dataset_id}: fetched {got:,} rows…")
        if n < page: break
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

def socrata_fetch_geojson(dataset_id, limit=50000, verbose=True):
    """
    Pull ALL features from a Socrata GeoJSON endpoint (.geojson). Returns a GeoDataFrame (EPSG:4326).
    Useful for polygon layers like census tracts.
    """
    feats, offset = [], 0
    while True:
        url = f"{BASE.format(id=dataset_id)}.geojson?$limit={limit}&$offset={offset}"
        r = requests.get(url, headers=_headers(), timeout=180); r.raise_for_status()
        gj = r.json(); chunk = gj.get("features", [])
        if not chunk: break
        feats.extend(chunk); offset += len(chunk)
        if verbose: print(f"{dataset_id} (geojson): fetched {offset:,} features…")
        if len(chunk) < limit: break
    return gpd.GeoDataFrame.from_features(feats, crs="EPSG:4326") if feats else gpd.GeoDataFrame(geometry=[], crs="EPSG:4326")

def add_lat_lon_from_location(df, location_col="location"):
    """
    Unpack Socrata 'location' Point dict into numeric lat/lon, preserving all other columns.
    """
    if location_col not in df.columns: return df
    def _lat(v):
        if isinstance(v, dict) and "latitude" in v: return v["latitude"]
        if isinstance(v, dict) and "coordinates" in v: return v["coordinates"][1]
    def _lon(v):
        if isinstance(v, dict) and "longitude" in v: return v["longitude"]
        if isinstance(v, dict) and "coordinates" in v: return v["coordinates"][0]
    out = df.copy()
    out["lat"] = pd.to_numeric(out[location_col].map(_lat), errors="coerce")
    out["lon"] = pd.to_numeric(out[location_col].map(_lon), errors="coerce")
    return out

def convert_datetime_columns(df, extra_force=()):
    """
    Convert columns that look like dates/times (by name) to pandas datetime.
    extra_force: iterable of column names to always attempt.
    """
    df = df.copy()
    name_pat = re.compile(r"(date|time|_at|_on)", re.I)
    candidates = [c for c in df.columns if name_pat.search(c)]
    candidates = sorted(set(candidates).union(extra_force))
    for c in candidates:
        df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

def classify_work_orders(works):
    """
    Add boolean flags for 'is_tree_removal' and 'is_tree_planting' + pick a best 'event_date'.
    Uses wotype + wstatus. Tighten patterns later if you discover coded fields.
    """
    w = works.copy()
    def s(x): return x.astype(str).str.lower() if isinstance(x, pd.Series) else pd.Series([], dtype=str)
    wotype, wstatus = s(w.get("wotype")), s(w.get("wstatus"))

    is_closed   = wstatus.isin({"closed","complete","completed"})
    is_removal  = wotype.str.contains(r"\btree removal\b", na=False) | \
                  wotype.str.contains(r"\btree removal for tree planting\b", na=False) | \
                  wotype.str.contains(r"\btree down\b", na=False)
    is_planting = wotype.str.contains(r"\btree plant\b", na=False) | \
                  wotype.str.contains(r"\bplanting\b", na=False)

    w["is_tree_removal"]  = is_removal & is_closed
    w["is_tree_planting"] = is_planting & is_closed

    # Choose a best event date per row: prefer completion/closed → else earliest parsed date on row
    date_priority = [c for c in [
        "completeddate","closeddate","finishdate","completion_date","date_completed",
        "actual_end_date","actual_finish_date"
    ] if c in w.columns]
    dt_cols = [c for c in w.columns if pd.api.types.is_datetime64_any_dtype(w[c])]

    def pick_event_date(row):
        for c in date_priority:
            v = row.get(c)
            if pd.notna(v): return v
        vals = [row[c] for c in dt_cols if pd.notna(row[c])]
        return min(vals) if vals else pd.NaT

    w["event_date"] = w.apply(pick_event_date, axis=1)
    return w


In [3]:
# --- Forestry Tree Points (all columns; keep only rows with geometry) ---
points_raw = socrata_fetch_tabular(
    DS_TREE_POINTS,
    where="location IS NOT NULL",
    select=None,
    limit=25000,
    max_rows=None
)
points = add_lat_lon_from_location(points_raw, "location").dropna(subset=["lat","lon"])
points = convert_datetime_columns(points, extra_force=("createddate", "updateddate"))
# -----------------------------------------------------------------------------------------------------
# Use 'createddate' as baseline start; we can refine with planting WOs later if earlier
if "createddate" not in points.columns:
    raise KeyError("Expected 'createddate' in Tree Points. Show points.columns if it's named differently.")

# Planting-space key in Tree Points
key_points = "plantingspaceglobalid" if "plantingspaceglobalid" in points.columns else \
             "planting_space_global_id" if "planting_space_global_id" in points.columns else None
if not key_points:
    raise KeyError("Expected a planting space id in Tree Points (e.g., 'plantingspaceglobalid').")

# --- Forestry Work Orders (all columns) ---
works_raw = socrata_fetch_tabular(
    DS_WORK_ORDERS,
    where=None,
    select=None,
    limit=25000,
    max_rows=None
)
works = convert_datetime_columns(works_raw)
works = classify_work_orders(works)

# Planting-space key in Work Orders
key_works = "plantingspaceglobalid" if "plantingspaceglobalid" in works.columns else \
            "planting_space_global_id" if "planting_space_global_id" in works.columns else None
if not key_works:
    print("⚠️ No planting-space key found in Work Orders; one-to-many join will be skipped.")

# --- (Optional) 2015 Tree Census for enrichment (species, DBH cross-check, etc.) ---
# You can use this later for QA or to backfill attributes where Tree Points are missing.
census15 = socrata_fetch_tabular(
    DS_CENSUS_2015,
    where="latitude IS NOT NULL AND longitude IS NOT NULL",
    select=None,
    limit=25000,
    max_rows=200_000  # start smaller; set None to attempt full
)
census15 = convert_datetime_columns(census15)


hn5i-inap: fetched 25,000 rows…
hn5i-inap: fetched 50,000 rows…
hn5i-inap: fetched 75,000 rows…
hn5i-inap: fetched 100,000 rows…
hn5i-inap: fetched 125,000 rows…
hn5i-inap: fetched 150,000 rows…
hn5i-inap: fetched 175,000 rows…
hn5i-inap: fetched 200,000 rows…
hn5i-inap: fetched 225,000 rows…
hn5i-inap: fetched 250,000 rows…
hn5i-inap: fetched 275,000 rows…
hn5i-inap: fetched 300,000 rows…
hn5i-inap: fetched 325,000 rows…
hn5i-inap: fetched 350,000 rows…
hn5i-inap: fetched 375,000 rows…
hn5i-inap: fetched 400,000 rows…
hn5i-inap: fetched 425,000 rows…
hn5i-inap: fetched 450,000 rows…
hn5i-inap: fetched 475,000 rows…
hn5i-inap: fetched 500,000 rows…
hn5i-inap: fetched 525,000 rows…
hn5i-inap: fetched 550,000 rows…
hn5i-inap: fetched 575,000 rows…
hn5i-inap: fetched 600,000 rows…
hn5i-inap: fetched 625,000 rows…
hn5i-inap: fetched 650,000 rows…
hn5i-inap: fetched 675,000 rows…
hn5i-inap: fetched 700,000 rows…
hn5i-inap: fetched 725,000 rows…
hn5i-inap: fetched 750,000 rows…
hn5i-inap: fe

In [24]:
points.columns

Index(['objectid', 'dbh', 'tpstructure', 'tpcondition',
       'plantingspaceglobalid', 'geometry', 'globalid', 'genusspecies',
       'createddate', 'location', 'stumpdiameter', 'updateddate', 'riskrating',
       'riskratingdate', 'planteddate', 'lat', 'lon', 'start_date', 'psid_x',
       'first_planting_date', 'psid_y', 'removal_date_x', 'exists_2010_2017',
       'psid', 'removal_date_y'],
      dtype='object')

In [29]:
pd.options.display.max_columns = 500

In [73]:
cut_created_max = pd.Timestamp("2017-12-31")   # keep created <= this (exclude NaT and > 2017-12-31)
cut_planted_max = pd.Timestamp("2009-12-31")   # accept planted <= this OR NaT

# --- Masks per your rules ---
# 1) CREATEDDATE: keep only rows with a non-null createddate on/before 2017-12-31
mask_created = points["createddate"].isna() | (points["createddate"] <= cut_created_max)

# 2) PLANTEDDATE: keep rows where planteddate is NaT OR planteddate <= 2009-12-31
mask_planted = points["planteddate"].isna() | (points["planteddate"] <= cut_planted_max)

# --- Apply both filters ---
points_filtered = points.loc[mask_created & mask_planted].copy()

In [74]:
#Only Alive Trees
points_filter = points_filtered[points_filtered['tpcondition']!='Dead']
points_filter = points_filter[~points_filter['tpstructure'].isin(['Stump - Uprooted','Stump'])]

In [75]:
points_filter

,objectid,dbh,tpstructure,tpcondition,plantingspaceglobalid,geometry,globalid,genusspecies,createddate,location,stumpdiameter,updateddate,riskrating,riskratingdate,planteddate,lat,lon,start_date,exists_2010_2017
0,230120,0,Retired,Unknown,B9DDFFE7-7387-4923-91EA-6E9212AA324F,POINT(-73.72963593901264 40.69403944537182),039E4FD7-CFB6-4518-9E16-1E37D10C994A,Unknown - Unknown,2015-06-19 10:39:00,"{'type': 'Point', 'coordinates': [-73.72963593...",NaN,NaT,NaN,NaT,NaT,40.694039,-73.729636,2015-06-19 10:39:00,True
3,746627,18,Full,Good,CCD7CEB4-D7FC-427F-982C-55355AB0989D,POINT(-73.93851920790104 40.60738960999758),FF71F967-C0E7-478E-BD3A-C54A2927A624,Acer - maple,2015-09-04 14:54:00,"{'type': 'Point', 'coordinates': [-73.93851920...",NaN,NaT,NaN,NaT,NaT,40.607390,-73.938519,2015-09-04 14:54:00,True
5,1058526,14,Full,Good,12AA0ADA-517C-4726-905A-E0A5A98EBCC2,POINT(-73.94958994062151 40.72358613049737),7AD3859E-4A1C-4C72-A4B1-F0AC2A97DB0E,Pinus - pine,2015-10-01 14:06:48,"{'type': 'Point', 'coordinates': [-73.94958994...",0,NaT,NaN,NaT,NaT,40.723586,-73.949590,2015-10-01 14:06:48,True
6,1392285,2,Full,Good,560979A5-0B59-46A5-A16E-8FDE0A01875D,POINT(-73.96235422914798 40.7946082222144),ECF53949-832B-4D61-B3C1-DD3042031E87,Quercus palustris - pin oak,2015-10-29 12:29:49,"{'type': 'Point', 'coordinates': [-73.96235422...",0,NaT,NaN,NaT,NaT,40.794608,-73.962354,2015-10-29 12:29:49,True
7,1415194,6,Full,Good,20EE5761-8B78-45D5-8EEE-81C335E5384E,POINT(-73.76501171400223 40.75057879438073),25DF6E74-6BED-45DC-978B-EAF670E0F365,Morus - mulberry,2015-10-29 13:49:57,"{'type': 'Point', 'coordinates': [-73.76501171...",0,NaT,NaN,NaT,NaT,40.750579,-73.765012,2015-10-29 13:49:57,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904320,5782313,3,Full,Good,5BB1F270-B6E0-4EB0-A998-E528605E93DF,POINT(-73.94682076785342 40.585317039825405),0598CDAF-0DEC-4E5D-BC0A-90831E9DF202,Pyrus calleryana 'Chanticleer' - 'Chanticleer'...,2017-12-11 00:00:00,"{'type': 'Point', 'coordinates': [-73.94682076...",NaN,2019-01-03 15:47:07,NaN,NaT,NaT,40.585317,-73.946821,2017-12-11 00:00:00,True
911047,5787952,3,Full,Good,71BD3EDA-6B99-45CC-8602-192D43C0548C,POINT(-73.8228103286091 40.70357559309544),77AC65C0-F876-443F-A59A-5179623A45C0,Zelkova serrata 'Mushashino' - 'Mushashino' Ze...,2017-05-17 00:00:00,"{'type': 'Point', 'coordinates': [-73.82281032...",NaN,2019-08-07 18:49:56,3,2019-07-30 14:34:32,NaT,40.703576,-73.822810,2017-05-17 00:00:00,True
912251,5787133,3,Full,Good,A2DD01E4-827D-4551-BA8B-7897F33E059E,POINT(-73.94297748515692 40.58612917453949),932FCCA1-CF52-4733-953F-54C4B05E50D5,Ginkgo biloba 'Autumn Gold' - 'Autumn Gold' Gi...,2017-12-11 00:00:00,"{'type': 'Point', 'coordinates': [-73.94297748...",NaN,2018-12-31 16:07:28,NaN,NaT,NaT,40.586129,-73.942977,2017-12-11 00:00:00,True
920885,5787679,4,Retired,Good,964DB6FF-08AF-426B-A30C-B13802F9EF28,POINT(-73.9962117329861 40.758407615772356),CFEFAF62-5A0E-4D47-9884-06044DA376D0,Ulmus parvifolia - Chinese elm,2017-06-21 00:00:00,"{'type': 'Point', 'coordinates': [-73.99621173...",NaN,2022-01-31 19:09:00,7,2021-11-09 17:16:00,NaT,40.758408,-73.996212,2017-06-21 00:00:00,True


In [81]:
closed_orders["createddate"].iloc[0]

'2019-12-13T15:46:00'

In [93]:
import pandas as pd

# 1) Pick the right status column and filter to CLOSED work orders
status_col = "wostatus" if "wostatus" in works_raw.columns else "wstatus"
closed_orders = works_raw.loc[
    works_raw[status_col].astype(str).str.lower().eq("closed")
].copy()

# 2) Parse dates
closed_orders["createddate"] = pd.to_datetime(closed_orders["createddate"], errors="coerce")
closed_orders["createddate"] = pd.to_datetime(closed_orders["createddate"], errors="coerce")
closed_orders["closeddate"] = pd.to_datetime(closed_orders["closeddate"], errors="coerce")


# 3) Keep rows with createddate < 2018-01-01 OR createddate is NaT
cut_off = pd.Timestamp("2018-01-01")
mask = ((closed_orders["createddate"].isna()) | (closed_orders["createddate"] < cut_off)
       &((closed_orders["createddate"].isna()) | (closed_orders["createddate"] < cut_off))
       &(closed_orders["closeddate"].isna() | (closed_orders["closeddate"] < cut_off)))
closed_orders_2017 = closed_orders.loc[mask].copy()
closed_orders_2017

print(len(closed_orders), "closed orders total")
print(len(closed_orders_2017), "closed orders with createddate < 2018-01-01 or NaT")

857919 closed orders total
223022 closed orders with createddate < 2018-01-01 or NaT


In [97]:
removals = closed_orders_2017[((closed_orders_2017["wotype"].astype(str).str.lower().str.contains("removal"))|
    (closed_orders_2017["wocategory"].astype(str).str.lower().str.contains("removal")))]

In [105]:
### saving files
points_filter.to_csv("tree_points_filters.csv",index=False)
removals.to_csv("tree_removals.csv",index=False)

In [104]:
points_filter[points_filter['objectid'].isin(removals['objectid'].unique())]

,objectid,dbh,tpstructure,tpcondition,plantingspaceglobalid,geometry,globalid,genusspecies,createddate,location,stumpdiameter,updateddate,riskrating,riskratingdate,planteddate,lat,lon,start_date,exists_2010_2017
16913,572710,14,Full,Good,EB020748-4A81-49CD-AB90-4E30FC5D4B18,POINT(-73.9598943947089 40.7059753264021),5698E64C-D5C2-442D-B1C6-9CBBFE548421,Tilia cordata - littleleaf linden,2015-08-25 10:30:18,"{'type': 'Point', 'coordinates': [-73.95989439...",0,2019-04-23 12:58:08,6,2019-04-23 08:58:07,NaT,40.705975,-73.959894,2015-08-25 10:30:18,True
17027,576324,10,Full,Good,405F02F9-EE29-47BE-8E49-F387D1A0BAE7,POINT(-73.9548039036249 40.80038406909359),A768A46B-6DAD-4BC9-9700-B09753800C36,Ginkgo biloba - maidenhair tree,2015-08-25 10:39:47,"{'type': 'Point', 'coordinates': [-73.95480390...",NaN,2023-04-04 15:24:41,5,2023-04-04 15:24:41,NaT,40.800384,-73.954804,2015-08-25 10:39:47,True
17080,579136,16,Full,Good,4057FFE5-75A2-4523-8494-866E1C4AE421,POINT(-73.87529095107415 40.87943160137904),47613EC6-9C23-42EA-BD7F-F2C20F8E6AD5,Ulmus americana - American elm,2015-08-25 10:47:37,"{'type': 'Point', 'coordinates': [-73.87529095...",NaN,2022-02-12 03:26:46,9,2022-02-12 03:26:46,NaT,40.879432,-73.875291,2015-08-25 10:47:37,True
17086,570686,3,Full,Fair,596A806C-4CBD-4383-8C1E-13146CF8F3E3,POINT(-73.99126906815867 40.76684875004094),AFF6345A-E78E-4B9F-8C8B-4B43DCEBE3E3,Quercus bicolor - swamp white oak,2015-08-25 10:24:58,"{'type': 'Point', 'coordinates': [-73.99126906...",NaN,2025-07-09 17:36:40,4,2025-07-09 17:36:40,NaT,40.766849,-73.991269,2015-08-25 10:24:58,True
17098,575532,6,Full,Good,3DF6A33F-86AC-49CA-9B73-DB8A9206663D,POINT(-74.08262926389015 40.629099609638374),1DC512B6-9C0C-494B-9660-4DA0A6F9F610,Prunus cerasifera - cherry plum,2015-08-25 10:37:38,"{'type': 'Point', 'coordinates': [-74.08262926...",NaN,2024-11-11 17:09:10,4,2024-11-11 17:09:10,NaT,40.629100,-74.082629,2015-08-25 10:37:38,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816765,5137639,5,Full,Good,60AEDDFB-A08A-4FBF-BB58-BE64ACB27104,POINT(-73.88369046775017 40.72094526219535),60AEDDFB-A08A-4FBF-BB58-BE64ACB27104,Prunus cerasifera - cherry plum,2017-12-20 11:11:11,"{'type': 'Point', 'coordinates': [-73.88369046...",0,NaT,NaN,NaT,NaT,40.720945,-73.883690,2017-12-20 11:11:11,True
817089,5128838,1,Full,Good,329B9A08-6482-48D1-9229-61538B5B2B44,POINT(-73.99853775186438 40.594703763052394),362E3BBA-76A3-49E1-844C-531EAB17E4FB,Maackia amurensis - Amur maackia,2017-12-05 09:46:31,"{'type': 'Point', 'coordinates': [-73.99853775...",NaN,2017-12-05 09:46:37,NaN,NaT,NaT,40.594704,-73.998538,2017-12-05 09:46:31,True
817281,5116021,1,Full,Good,27868400-117C-4728-B36E-AE9DBAA85FF2,POINT(-73.99775304492097 40.597036868550795),54C40110-25A5-459B-956B-7867F051B5A6,Acer campestre - hedge maple,2017-11-15 09:28:14,"{'type': 'Point', 'coordinates': [-73.99775304...",NaN,2017-11-15 09:28:29,NaN,NaT,NaT,40.597037,-73.997753,2017-11-15 09:28:14,True
817687,5118033,3,Retired,Excellent,F91E2AA7-3E8D-4492-85DB-94E5921DE7B8,POINT(-73.94483814199175 40.825810276150186),3D08FAC2-A379-497F-A52B-B30792A3B98C,Unknown - Unknown,2017-11-22 11:23:32,"{'type': 'Point', 'coordinates': [-73.94483814...",NaN,2021-10-18 15:09:58,NaN,NaT,NaT,40.825810,-73.944838,2017-11-22 11:23:32,True


In [86]:
closed_orders_2017.columns


Index(['objectid', 'wotype', 'wostatus', 'boroughcode', 'communityboard',
       'buildingnumber', 'streetname', 'locationdetails', 'wocontract',
       'parkname', 'wocategory', 'wowoodremains', 'sanitationzone',
       'inspectionglobalid', 'globalid', 'wowireconflict', 'geometry',
       'createddate', 'woentity', 'recommendedspecies', 'nta', 'location',
       ':@computed_region_efsh_h5xi', ':@computed_region_f5dn_yrer',
       ':@computed_region_yeji_bk3q', ':@computed_region_92fq_4b7q',
       ':@computed_region_sbqj_enih', 'wopriority', 'sanitationassigneddate',
       'updateddate', 'parkzone', 'zipcode', 'citycouncil', 'statesenate',
       'stateassembly', 'congressional', 'crossstreet1', 'crossstreet2',
       'actualfinishdate', 'closeddate', 'woproject', 'cancelreason',
       'canceldate', 'latitude', 'longitude', 'census_tract', 'sidewalkdamage',
       'physicalid', 'woequipment', 'bin', 'bbl', 'crewglobalid',
       'projstartdate', 'sanitationremovaldate', 'sanitation

# ------------------------

In [22]:
# === Robust lifecycle & presence using Forestry Tree Points schema (with planteddate) ===
import pandas as pd
import re

WINDOW_START = pd.Timestamp("2010-01-01")
WINDOW_END   = pd.Timestamp("2017-12-31")
YEARS = list(range(2010, 2018))

# --- Ensure the expected columns are present (from your schema screenshot) ---
needed_cols = ["planteddate", "createddate", "updateddate", "plantingspaceglobalid", "location"]
missing = [c for c in needed_cols if c not in points.columns]
if missing:
    raise KeyError(f"Tree Points is missing expected columns: {missing}")

# --- 1) Coerce Tree Points dates (exact field names) ---
for c in ["planteddate", "createddate", "updateddate", "riskratingdate"]:
    if c in points.columns:
        points[c] = pd.to_datetime(points[c], errors="coerce")

# --- 2) Prefer planteddate, else createddate ---
points["start_date"] = points[["planteddate", "createddate"]].min(axis=1)

# --- 3) Build removal dates from Work Orders (earliest CLOSED removal per space) ---
def s(x): return x.astype(str).str.lower() if isinstance(x, pd.Series) else pd.Series([], dtype=str)

# Coerce WO datetimes
date_like = [c for c in works.columns if re.search(r"(date|time|_at|_on)", c, re.I)]
for c in date_like:
    works[c] = pd.to_datetime(works[c], errors="coerce")

# Flags (tighten to your schema if needed)
wotype  = s(works.get("wotype"))
wstatus = s(works.get("wstatus"))
is_closed   = wstatus.isin({"closed","complete","completed"})
is_removal  = (
    wotype.str.contains(r"\btree removal\b", na=False) |
    wotype.str.contains(r"\btree removal for tree planting\b", na=False) |
    wotype.str.contains(r"\btree down\b", na=False)
)

# Choose a best event date per WO row (prefer completion/closed fields)
date_priority = [c for c in [
    "completeddate","closeddate","finishdate","completion_date","date_completed",
    "actual_end_date","actual_finish_date"
] if c in works.columns]
dt_cols = [c for c in works.columns if pd.api.types.is_datetime64_any_dtype(works[c])]

def pick_event_date(row):
    for c in date_priority:
        v = row.get(c)
        if pd.notna(v): return v
    vals = [row[c] for c in dt_cols if pd.notna(row[c])]
    return min(vals) if vals else pd.NaT

works["event_date"] = works.apply(pick_event_date, axis=1)

# Ensure planting-space key name
key_points = "plantingspaceglobalid"
key_works  = "plantingspaceglobalid" if "plantingspaceglobalid" in works.columns else None

if key_works:
    removal_dates = (
        works.loc[is_removal & is_closed & works["event_date"].notna(), [key_works, "event_date"]]
             .sort_values("event_date")
             .drop_duplicates(subset=[key_works], keep="first")
             .rename(columns={key_works: "psid", "event_date": "removal_date"})
    )
    points = points.merge(removal_dates, left_on=key_points, right_on="psid", how="left")
else:
    points["removal_date"] = pd.NaT  # no WO join possible

# --- 4) Keep only spaces that overlap the 2010–2017 window ---
def overlaps_window(start, end, t0=WINDOW_START, t1=WINDOW_END):
    if pd.isna(start): return False
    if start > t1:     return False
    if pd.isna(end):   return True
    return end >= t0

points["exists_2010_2017"] = points.apply(lambda r: overlaps_window(r["start_date"], r["removal_date"]), axis=1)
trees_2010_2017 = points.loc[points["exists_2010_2017"]].copy()

# --- 5) Presence flags (vectorized) ---
s = pd.to_datetime(trees_2010_2017["start_date"])
e = pd.to_datetime(trees_2010_2017["removal_date"])
s_ok = s.notna()

for y in YEARS:
    asof = pd.Timestamp(f"{y}-12-31")
    trees_2010_2017[f"present_{y}"] = (s_ok & (s <= asof) & (e.isna() | (e > asof))).values

# --- 6) Quick sanity reports ---
print("Rows in trees_2010_2017:", len(trees_2010_2017))
print("start_date min/max:", trees_2010_2017["start_date"].min(), trees_2010_2017["start_date"].max())
print("removal_date min/max:", trees_2010_2017["removal_date"].min(), trees_2010_2017["removal_date"].max())
print({y: int(trees_2010_2017[f'present_{y}'].sum()) for y in YEARS})


KeyError: 'removal_date'

In [4]:
# ---- Start date per planting space ----
# baseline = createddate
points["start_date"] = points["createddate"].copy()

# Optional: earlier planting WO can predate createddate → choose the earlier one
if key_works:
    plant_dates = (
        works.loc[works["is_tree_planting"] & works["event_date"].notna(), [key_works, "event_date"]]
             .sort_values("event_date")
             .drop_duplicates(subset=[key_works], keep="first")
             .rename(columns={key_works: "psid", "event_date": "first_planting_date"})
    )
    points = points.merge(plant_dates, left_on=key_points, right_on="psid", how="left")
    points["start_date"] = points[["start_date","first_planting_date"]].min(axis=1)

# ---- Removal date per planting space ----
removals = pd.DataFrame(columns=["psid","removal_date"])
if key_works:
    removals = (
        works.loc[works["is_tree_removal"] & works["event_date"].notna(), [key_works, "event_date"]]
             .sort_values("event_date")
             .drop_duplicates(subset=[key_works], keep="first")
             .rename(columns={key_works: "psid", "event_date": "removal_date"})
    )
points = points.merge(removals, left_on=key_points, right_on="psid", how="left")

# ---- Keep trees that existed at ANY point in 2010..2017 ----
# i.e., started on/before 2017-12-31 and (no removal) OR (removal after 2010-01-01)
window_start = pd.Timestamp("2010-01-01")
window_end   = pd.Timestamp("2017-12-31")

def overlaps_window(start, end, t0=window_start, t1=window_end):
    # presence interval = [start, end) where end is removal_date or +∞
    if pd.isna(start) or start > t1:
        return False  # begins after the window
    if pd.isna(end):
        return True   # still present -> overlaps
    return end >= t0  # removed after window start -> overlaps

points["exists_2010_2017"] = points.apply(lambda r: overlaps_window(r["start_date"], r["removal_date"]), axis=1)
trees_2010_2017 = points.loc[points["exists_2010_2017"]].copy()

print(f"Trees overlapping 2010–2017: {len(trees_2010_2017):,} of {len(points):,}")


Trees overlapping 2010–2017: 815,399 of 1,093,439


In [21]:
trees_2010_2017['present_2015'].unique()

array([ True, False])

In [15]:
trees_2010_2017['present_2010'].unique()

array([False])

In [5]:
def present_as_of(row, as_of):
    start, end = row.get("start_date"), row.get("removal_date")
    if pd.isna(start) or start > as_of: return False
    if pd.notna(end) and end <= as_of:  return False
    return True

for y, dt in AS_OF.items():
    trees_2010_2017[f"present_{y}"] = trees_2010_2017.apply(present_as_of, axis=1, as_of=dt)

# A long, analysis-friendly table: one row per planting space per year with presence flag
presence_long = (
    trees_2010_2017[["plantingspaceglobalid","lat","lon","start_date","removal_date"] + [c for c in trees_2010_2017.columns if c.startswith("present_")]]
    .rename(columns={key_points: "psid"})
    .melt(
        id_vars=["psid","lat","lon","start_date","removal_date"],
        var_name="present_year",
        value_name="present"
    )
    .assign(year=lambda d: d["present_year"].str.replace("present_","").astype(int))
    .drop(columns=["present_year"])
)

# Keep only the 2010..2017 range (already is, but future-proof)
presence_long = presence_long.loc[presence_long["year"].between(2010, 2017)].reset_index(drop=True)
presence_long.head()


,psid,lat,lon,start_date,removal_date,present,year
0,B9DDFFE7-7387-4923-91EA-6E9212AA324F,40.694039,-73.729636,2015-06-19 10:39:00,NaT,False,2010
1,6BC4E975-AB95-4473-875D-E00155F17AF7,40.671521,-73.970912,2015-05-29 14:53:00,NaT,False,2010
2,7B2D2E72-5564-4975-ABCA-7627241F6667,40.747909,-73.954832,2015-08-25 10:44:33,NaT,False,2010
3,CCD7CEB4-D7FC-427F-982C-55355AB0989D,40.607390,-73.938519,2015-09-04 14:54:00,NaT,False,2010
4,12AA0ADA-517C-4726-905A-E0A5A98EBCC2,40.723586,-73.949590,2015-10-01 14:06:48,NaT,False,2010


In [10]:
presence_long[~presence_long['removal_date'].isnull()]

,psid,lat,lon,start_date,removal_date,present,year


In [7]:
# --- One-to-many: Tree Points × Work Orders (robust preview) ---

import re

# 1) Merge (one tree/planting space -> many work orders)
if key_works:
    tree_work_long = trees_2010_2017.merge(
        works,
        left_on=key_points,
        right_on=key_works,
        how="left",
        suffixes=("", "_wo"),  # WO columns get _wo only if a name collides with left frame
    )
else:
    tree_work_long = trees_2010_2017.copy()  # no WO key available

print("tree_work_long shape:", tree_work_long.shape)

# 2) Small resolver so we don't assume exact column spellings or suffixes
def find_col(df, candidates):
    """
    Return the first matching column name in df for any candidate.
    - Case-insensitive
    - Tries '<candidate>_wo' too (in case merge added suffix)
    - Falls back to "ignore-underscores" matching
    """
    lower = {c.lower(): c for c in df.columns}
    for cand in candidates:
        key = cand.lower()
        if key in lower:
            return lower[key]
        if f"{key}_wo" in lower:
            return lower[f"{key}_wo"]
    # loose pass: ignore underscores
    norm = {c.lower().replace("_",""): c for c in df.columns}
    for cand in candidates:
        key = cand.lower().replace("_","")
        if key in norm:
            return norm[key]
    return None

col_psid      = find_col(tree_work_long, [key_points, "plantingspaceglobalid", "planting_space_global_id"])
col_wotype    = find_col(tree_work_long, ["wotype", "wo_type", "worktype", "work_type"])
col_wstatus   = find_col(tree_work_long, ["wstatus", "status", "wo_status", "workstatus"])
col_is_remove = find_col(tree_work_long, ["is_tree_removal"])
col_is_plant  = find_col(tree_work_long, ["is_tree_planting"])
col_event     = find_col(tree_work_long, ["event_date"])

print("Resolved columns:", {
    "psid": col_psid,
    "wotype": col_wotype,
    "wstatus": col_wstatus,
    "is_tree_removal": col_is_remove,
    "is_tree_planting": col_is_plant,
    "event_date": col_event,
})

# 3) Preview only the columns that actually exist
preview_cols = [c for c in [col_psid, col_wotype, col_wstatus, col_is_remove, col_is_plant, col_event] if c]
display(tree_work_long[preview_cols].head(10))

# If you're curious which "status/type" columns are present, uncomment:
# print(sorted([c for c in tree_work_long.columns if "status" in c.lower()]))
# print(sorted([c for c in tree_work_long.columns if "type" in c.lower()]))


tree_work_long shape: (815403, 90)
Resolved columns: {'psid': 'plantingspaceglobalid', 'wotype': 'wotype', 'wstatus': 'wostatus', 'is_tree_removal': 'is_tree_removal', 'is_tree_planting': 'is_tree_planting', 'event_date': 'event_date'}


,plantingspaceglobalid,wotype,wostatus,is_tree_removal,is_tree_planting,event_date
0,B9DDFFE7-7387-4923-91EA-6E9212AA324F,NaN,NaN,NaN,NaN,NaT
1,6BC4E975-AB95-4473-875D-E00155F17AF7,NaN,NaN,NaN,NaN,NaT
2,7B2D2E72-5564-4975-ABCA-7627241F6667,NaN,NaN,NaN,NaN,NaT
3,CCD7CEB4-D7FC-427F-982C-55355AB0989D,NaN,NaN,NaN,NaN,NaT
4,12AA0ADA-517C-4726-905A-E0A5A98EBCC2,NaN,NaN,NaN,NaN,NaT
5,560979A5-0B59-46A5-A16E-8FDE0A01875D,NaN,NaN,NaN,NaN,NaT
6,20EE5761-8B78-45D5-8EEE-81C335E5384E,NaN,NaN,NaN,NaN,NaT
7,65D01FA8-80D0-4F72-9211-F65FC3F32D0A,NaN,NaN,NaN,NaN,NaT
8,F69631A4-8705-46BC-85D7-037CBF4BADA3,NaN,NaN,NaN,NaN,NaT
9,3D063EB9-5F20-4E3C-98AC-63C4ECE0A935,NaN,NaN,NaN,NaN,NaT


In [ ]:
if CT_DATASET_ID:
    tracts = socrata_fetch_geojson(CT_DATASET_ID).to_crs(TARGET_CRS)

    # Choose some tract id columns if present
    tract_id_cols = [c for c in ["ct2010","boroct2010","boroname","geoid","geoid10"] if c in tracts.columns]
    ct_keep = tracts[tract_id_cols + ["geometry"]] if tract_id_cols else tracts[["geometry"]].copy()

    # Trees → CTs
    gtrees = gpd.GeoDataFrame(
        trees_2010_2017,
        geometry=gpd.points_from_xy(trees_2010_2017["lon"], trees_2010_2017["lat"]),
        crs="EPSG:4326"
    ).to_crs(TARGET_CRS)

    gtrees_ct = gpd.sjoin(gtrees, ct_keep, how="left", predicate="within").drop(columns=["index_right"])
    trees_with_ct = pd.DataFrame(gtrees_ct.drop(columns="geometry"))

    # Each work order → CT by merging its tree's tract info
    if key_works:
        wo_with_ct = works.merge(
            trees_with_ct[[key_points] + tract_id_cols].drop_duplicates(key_points),
            left_on=key_works, right_on=key_points, how="left"
        )


In [ ]:
# 1) Master tree table (filtered to those overlapping 2010–2017), with start/removal dates and coords
trees_master = trees_2010_2017.copy()

# 2) Per-year presence long table (psid, year, present, lat, lon)
trees_presence_by_year = presence_long.copy()

# 3) Exploded one-to-many Tree x Work Order table
trees_x_workorders = tree_work_long.copy()

print("trees_master:", trees_master.shape)
print("trees_presence_by_year:", trees_presence_by_year.shape)
print("trees_x_workorders:", trees_x_workorders.shape)

# Example saves (optional)
# trees_master.to_parquet("trees_master_2010_2017.parquet", index=False)
# trees_presence_by_year.to_parquet("trees_presence_by_year.parquet", index=False)
# trees_x_workorders.to_parquet("trees_x_workorders.parquet", index=False)


In [ ]:
trees_master

In [ ]:
trees_presence_by_year

In [ ]:
trees_x_workorders